In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import string
import math

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17019368130090459604
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4815519744
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7549479406538207553
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
SEQUENCE_LEN = 60
BATCH_SIZE = 512
EPOCHS = 20
LSTM_DIM = 256
LAYER_COUNT = 4
DROPOUT = 0.2

In [5]:
with open("./data/complete_lotr.txt", "rb") as f:
    text = f.read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 2588993 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

Three Rings for the Elven-kings under the sky,
               Seven for the Dwarf-lords in their halls of stone,
            Nine for Mortal Men doomed to die,
              One for the Dark Lord on his dark throne
           In the Land of Mordo


In [7]:
#Generating Vocabulary, character to index  and index to character dictionary
vocab = sorted(set(text))
vocab_size = len(vocab)
char_to_idx = {c:i for i,c in enumerate(vocab)}
idx_to_char = {i:c for c,i in char_to_idx.items()}

print('{} unique characters'.format(len(vocab)))
print(char_to_idx.items())

97 unique characters
dict_items([('\t', 0), ('\n', 1), ('\r', 2), (' ', 3), ('!', 4), ('"', 5), ("'", 6), ('(', 7), (')', 8), ('*', 9), (',', 10), ('-', 11), ('.', 12), ('/', 13), ('0', 14), ('1', 15), ('2', 16), ('3', 17), ('4', 18), ('5', 19), ('6', 20), ('7', 21), ('8', 22), ('9', 23), (':', 24), (';', 25), ('=', 26), ('?', 27), ('A', 28), ('B', 29), ('C', 30), ('D', 31), ('E', 32), ('F', 33), ('G', 34), ('H', 35), ('I', 36), ('J', 37), ('K', 38), ('L', 39), ('M', 40), ('N', 41), ('O', 42), ('P', 43), ('Q', 44), ('R', 45), ('S', 46), ('T', 47), ('U', 48), ('V', 49), ('W', 50), ('X', 51), ('Y', 52), ('Z', 53), ('_', 54), ('`', 55), ('a', 56), ('b', 57), ('c', 58), ('d', 59), ('e', 60), ('f', 61), ('g', 62), ('h', 63), ('i', 64), ('j', 65), ('k', 66), ('l', 67), ('m', 68), ('n', 69), ('o', 70), ('p', 71), ('q', 72), ('r', 73), ('s', 74), ('t', 75), ('u', 76), ('v', 77), ('w', 78), ('x', 79), ('y', 80), ('z', 81), ('É', 82), ('Ó', 83), ('á', 84), ('â', 85), ('ä', 86), ('é', 87), ('ê', 

In [8]:
def batch_generator(text, batch_size = BATCH_SIZE, sequence_size = SEQUENCE_LEN, vocab_size = vocab_size):
    n_steps = (len(text)-sequence_size) // (batch_size)
    while True:
        for batch_idx in range(n_steps):
            x = np.zeros((batch_size,sequence_size,vocab_size))
            y = np.zeros((batch_size, vocab_size))

            batch_start = batch_idx*batch_size

            for sents_idx in range(batch_size):
              sent_start = batch_start + sents_idx

              for s in range(sequence_size):
                x[sents_idx, s, char_to_idx[text[sent_start + s]]] = 1
              y[sents_idx,char_to_idx[text[sent_start + s + 1]]] = 1
            
            yield x,y 

In [9]:
def build_model():
    model = tf.keras.models.Sequential()
    for i in range(LAYER_COUNT):
        model.add(
            tf.keras.layers.LSTM(LSTM_DIM,
                                 return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                                 input_shape = (SEQUENCE_LEN, vocab_size)
                                )
            )
        model.add(tf.keras.layers.Dropout(DROPOUT))
    model.add(tf.keras.layers.Dense(vocab_size,
                                    activation='softmax'
                                   ))
    adam = tf.keras.optimizers.Adam()
    
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

    

In [10]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                             min_delta=0, patience=3,
                                             verbose=0,
                                             mode='auto')
filepath = "model_weights_saved.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,
                                                monitor='loss', verbose=1, save_best_only=True, 
                                                mode='min')
desired_callbacks = [earlystop, checkpoint]
training_model = build_model()    
print(training_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 256)           362496    
_________________________________________________________________
dropout (Dropout)            (None, 60, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 256)           525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               5

In [11]:
history = training_model.fit_generator(
    batch_generator(text),
    steps_per_epoch = (len(text)-SEQUENCE_LEN) // (BATCH_SIZE),
    #max_queue_size=1, # no more than one queued batch in RAM
    epochs=EPOCHS,
    callbacks=desired_callbacks,
    initial_epoch=0
)

C:\Users\utsav\anaconda3\envs\NLP-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
5056/5056 [==============================] - 794s 156ms/step - loss: 2.4532 - accuracy: 0.3277

Epoch 00001: loss improved from inf to 2.02093, saving model to model_weights_saved.hdf5
Epoch 2/20
5056/5056 [==============================] - 800s 158ms/step - loss: 1.5703 - accuracy: 0.5362

Epoch 00002: loss improved from 2.02093 to 1.50747, saving model to model_weights_saved.hdf5
Epoch 3/20
5056/5056 [==============================] - 796s 157ms/step - loss: 1.4352 - accuracy: 0.5718

Epoch 00003: loss improved from 1.50747 to 1.39778, saving model to model_weights_saved.hdf5
Epoch 4/20
5056/5056 [==============================] - 796s 157ms/step - loss: 1.3699 - accuracy: 0.5886

Epoch 00004: loss improved from 1.39778 to 1.33982, saving model to model_weights_saved.hdf5
Epoch 5/20
5056/5056 [==============================] - 796s 157ms/step - loss: 1.3276 - accuracy: 0.5990

Epoch 00005: loss improved from 1.33982 to 1.30158, saving model to model_weights_saved.hdf5
Epoc

In [12]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
plot_graphs(history, 'loss')